In [57]:
%%capture
!pip install wikidataintegrator

In [64]:
import pandas as pd
from wikidataintegrator import wdi_core, wdi_login
import getpass
from datetime import datetime
import copy
import requests
import json

In [61]:
print("username:")
username = input()
print("password:")
password = getpass.getpass()
login = wdi_login.WDLogin(user=username, pwd=password)

username:
andrawaag
password:
········


In [2]:
df = pd.read_csv("fungi.csv")

In [3]:
df

,taxon,taxonLabel,commons,inaturalist,gbif,image
0,http://www.wikidata.org/entity/Q80705,Albatrellus subrubescens,https://commons.wikimedia.org/wiki/Category:Al...,350020.0,3357733.0,http://commons.wikimedia.org/wiki/Special:File...
1,http://www.wikidata.org/entity/Q111697,Chroogomphus helveticus,https://commons.wikimedia.org/wiki/Category:Ch...,344022.0,2524993.0,http://commons.wikimedia.org/wiki/Special:File...
2,http://www.wikidata.org/entity/Q131227,Amanita muscaria,https://commons.wikimedia.org/wiki/Amanita_mus...,48715.0,8168319.0,http://commons.wikimedia.org/wiki/Special:File...
3,http://www.wikidata.org/entity/Q19740,Boletus edulis,https://commons.wikimedia.org/wiki/Boletus_edulis,48701.0,5954958.0,http://commons.wikimedia.org/wiki/Special:File...
4,http://www.wikidata.org/entity/Q39119,Exidia glandulosa,https://commons.wikimedia.org/wiki/Exidia_glan...,55568.0,2553607.0,http://commons.wikimedia.org/wiki/Special:File...
...,...,...,...,...,...,...
1208,http://www.wikidata.org/entity/Q12955841,Xerocomus ferrugineus,https://commons.wikimedia.org/wiki/Category:Xe...,928737.0,2519484.0,http://commons.wikimedia.org/wiki/Special:File...
1209,http://www.wikidata.org/entity/Q15095332,Tricholoma stiparophyllum,https://commons.wikimedia.org/wiki/Category:Tr...,714557.0,5241748.0,http://commons.wikimedia.org/wiki/Special:File...
1210,http://www.wikidata.org/entity/Q26203731,Rubroboletus satanas,https://commons.wikimedia.org/wiki/Rubroboletu...,544554.0,9217295.0,http://commons.wikimedia.org/wiki/Special:File...
1211,http://www.wikidata.org/entity/Q26203742,Rubroboletus rhodoxanthus,https://commons.wikimedia.org/wiki/Category:Ru...,824010.0,7487519.0,http://commons.wikimedia.org/wiki/Special:File...


In [78]:
missingGbif = df[df["gbif"].isnull()]
gbifids = dict()
for index, row in missingGbif.iterrows():
    gbifids[row["taxon"]] = json.loads(requests.get("https://api.gbif.org/v1/species/match?name="+row["taxonLabel"].replace(" ", "+")).text)
    

In [86]:
# GBIF reference

refStatedIn = wdi_core.WDItemID(value="Q1531570", prop_nr="P248", is_reference=True) 
refRetrieved = wdi_core.WDTime(datetime.now().strftime("+%Y-%m-%dT00:00:00Z"), prop_nr="P813", is_reference=True)
reference = [refStatedIn, refRetrieved]

for taxon in gbifids.keys():
    print(str(gbifids[taxon]["usageKey"]),
      str(gbifids[taxon]["confidence"]))
    statement = [wdi_core.WDExternalID(value=str(gbifids[taxon]["usageKey"]), prop_nr="P846", references=[copy.deepcopy(reference)])]
    item = wdi_core.WDItemEngine(wd_item_id=taxon.replace("http://www.wikidata.org/entity/",""), data=statement,)
    print(print(item.write(login)))
    

8249431 98
Q21159167
None
3348530 97
Q81007567
None
5240292 97
Q105049169
None
8028942 97
Q105051865
None
7729510 97
Q105052026
None
2531735 97
Q105052806
None
10792755 98
Q107481496
None
8110284 98
Q107504319
None
9501413 98
Q107504322
None
8094873 98
Q107505497
None
10841791 98
Q107530823
None
8149072 98
Q107596984
None
8291787 98
Q107597063
None
7959945 98
Q107597460
None
8024986 98
Q107597548
None
8324783 98
Q107598123
None
7618220 98
Q107598154
None
8349253 98
Q107599241
None
7561134 98
Q107611661
None
9928082 98
Q108934934
None
8200672 98
Q107597572
None
8225851 97
Q80984088
None
8015843 97
Q80988439
None
3350510 97
Q80993086
None
7692557 98
Q81000107
None
3348919 97
Q81005025
None
10171641 98
Q108933624
None
8259749 98
Q46810615
None
8031042 98
Q54371194
None
9010424 97
Q20968163
None
2535402 97
Q11862369
None
3340325 97
Q80984943
None
8347123 98
Q81020679
None
7698408 98
Q81023992
None
2527866 94
Q81026004
None
2527169 97
Q81202028
None
8315818 98
Q84336744
None
7590943 98
Q105

In [79]:
print(str(gbifids['http://www.wikidata.org/entity/Q21159167']["usageKey"]),
      str(gbifids['http://www.wikidata.org/entity/Q21159167']["confidence"]))

8249431 98


In [38]:
missingiNaturalists = df[df["inaturalist"].isnull()]
inatids = dict()
for index, row in missingiNaturalists.iterrows():
    inatids[row["taxon"]] = json.loads(requests.get("https://api.inaturalist.org/v1/taxa/autocomplete?q="+row["taxonLabel"].replace(" ", "+")).text)


In [68]:
# inaturalist reference

refStatedIn = wdi_core.WDItemID(value="Q73616981", prop_nr="P248", is_reference=True) 
refRetrieved = wdi_core.WDTime(datetime.now().strftime("+%Y-%m-%dT00:00:00Z"), prop_nr="P813", is_reference=True)
reference = [refStatedIn, refRetrieved]

for taxon in inatids.keys():
    if inatids[taxon]["total_results"]>0:
        statement = [wdi_core.WDExternalID(value=str(record["id"]), prop_nr="P3151", references=[copy.deepcopy(reference)])]
        item = wdi_core.WDItemEngine(wd_item_id=taxon.replace("http://www.wikidata.org/entity/",""), data=statement)
        print(item.write(login))
        print(taxon, inatids[taxon]["results"][0]["matched_term"], )


Q21157112
http://www.wikidata.org/entity/Q21157112 Clitocybe obsoleta
Q21159167
http://www.wikidata.org/entity/Q21159167 Cortinarius fasciatus
Q10460204
http://www.wikidata.org/entity/Q10460204 Cortinarius scandens
Q81008809
http://www.wikidata.org/entity/Q81008809 Cortinarius russus
Q81011088
http://www.wikidata.org/entity/Q81011088 Cortinarius striaepilus
Q81014323
http://www.wikidata.org/entity/Q81014323 Cortinarius testaceofolius
Q81014939
http://www.wikidata.org/entity/Q81014939 Cortinarius turgidoides
Q81015635
http://www.wikidata.org/entity/Q81015635 Cortinarius vespertinus
Q84326493
http://www.wikidata.org/entity/Q84326493 Cortinarius impennoides
Q84821083
http://www.wikidata.org/entity/Q84821083 Cortinarius variecolor
Q105049169
http://www.wikidata.org/entity/Q105049169 Amanita gemmata
Q105052037
http://www.wikidata.org/entity/Q105052037 Cortinarius obtusus striatulus
Q107481496
http://www.wikidata.org/entity/Q107481496 Amaropostia stiptica
Q107504322
http://www.wikidata.org/e